In [23]:
import folium as fl
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import rasterio.features
import rasterio.warp
from rasterio.mask import mask
import os 
import numpy as np
import earthpy as et 

# fazer mapa das estacoes fluv

In [ ]:
# load metadata from ANA
#efluv = 'D:/ENS410064/Dados/brutos/ANA/metadata_ANA-flow_2023-10-03.txt' 
# get lat lon from used stations 

br = gpd.read_file(r'Z:\- GIS -\IBGE\SC_Municipios_2020\SC_Municipios_2020.shp')
#D:\GeoBase\IBGE\BR_Municipios_2020')
fig,ax = plt.subplots()
#br.plot(ax=ax)

br[br['SIGLA_UF']=='SC'].explore()
br[br['SIGLA_UF']=='SC'].explore('AREA_KM2')

In [40]:
srtm = rasterio.open(r'Z:\- GIS -\SRTM\SRTM_SC_UTM.tif')
srtm.width

6681

In [31]:
srtm.height

4114

In [32]:
{i: dtype for i, dtype in zip(srtm.indexes, srtm.dtypes)}

{1: 'float32'}

In [8]:
srtm.bounds

BoundingBox(left=215894.1758, bottom=6749106.6746, right=764486.2397, top=7129160.9218)

In [27]:
srtm.crs

CRS.from_epsg(31982)

In [10]:
srtm.indexes

(1,)

In [12]:
band1 = srtm.read(1)
band1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [37]:
shape1 = gpd.read_file(r'Z:\- GIS -\IBGE\vegetacao\VG_REG_FITO_UTM_SC.shp')
shape1.crs

<Projected CRS: EPSG:31982>
Name: SIRGAS 2000 / UTM zone 22S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 54°W and 48°W, northern and southern hemispheres, onshore and offshore. In remainder of South America - between 54°W and 48°W, southern hemisphere, onshore and offshore.
- bounds: (-54.0, -54.18, -47.99, 7.04)
Coordinate Operation:
- name: UTM zone 22S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [38]:
# extract the geometry in GeoJSON format
geoms = shape1.geometry.values # list of shapely geometries
geometry = geoms[0] # shapely geometry
# transform to GeJSON format
from shapely.geometry import mapping
geoms = [mapping(geoms[0])]

In [41]:
# extract the raster values within the polygon 
with srtm as src:
     out_image, out_transform = mask(src, geoms, crop=True)

In [36]:
# extract the values of the masked array
data = out_image.data[0]
# extract the row, columns of the valid values
import numpy as np
row, col = np.where(data != no_data) 
elev = np.extract(data != no_data, data)
from rasterio import Affine # or from affine import Affine
T1 = out_transform * Affine.translation(0.5, 0.5) # reference the pixel centre
rc2xy = lambda r, c: (c, r) * T1  


NotImplementedError: multi-dimensional sub-views are not implemented

In [ ]:

d = gpd.GeoDataFrame({'col':col,'row':row,'elev':elev})
# coordinate transformation
d['x'] = d.apply(lambda row: rc2xy(row.row,row.col)[0], axis=1)
d['y'] = d.apply(lambda row: rc2xy(row.row,row.col)[1], axis=1)
# geometry
from shapely.geometry import Point
d['geometry'] =d.apply(lambda row: Point(row['x'], row['y']), axis=1)
# first 2 points
d.head(2)
# save to a shapefile
d.to_file('result.shp', driver='ESRI Shapefile')

In [ ]:
with srtm as dataset:

    # Read the dataset's valid data mask as a ndarray.
    mask = dataset.dataset_mask()

    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(
            mask, transform=dataset.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(
            dataset.crs, 'EPSG:4326', geom, precision=6)

        # Print GeoJSON shapes to stdout.
        print(geom)
